In [ ]:
import torch
import os
import sys
sys.path.append("/content")


In [ ]:
rebuild_dataset = True
visualize_sample = True

In [ ]:
!nvidia-smi

In [ ]:
# Get RT-DETR
if not os.path.isdir("/content/RT-DETR"):
  %cd /content
  print("Cloning RT-DETR")
  !git clone https://github.com/lyuwenyu/RT-DETR.git
else:
  print("RT-DETR is already cloned.")

%cd RT-DETR/rtdetrv2_pytorch
!pip install --upgrade pip -q
!pip install -r requirements.txt -q
%cd ..

# Get pretrained weights <- starting training from here
if not os.path.isfile("/content/RT-DETR/pretrained/rtdetrv2_r18vd_120e_coco.pth"):
  !mkdir -p pretrained
  !wget -P pretrained https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetrv2_r18vd_120e_coco.pth

In [ ]:
# Copy and unzip data from GDrive
%cd /content
if not os.path.isdir("/content/soccertrack") and rebuild_dataset:
  if not os.path.isfile("/content/soccertrack.zip"):
    print('Copying soccertrack dataset ...')
    from google.colab import drive
    drive.mount('/content/drive')
    !cp /content/drive/MyDrive/datasets/soccertrack.zip .
  print('Unzipping soccertrack dataset ...')
  !unzip -q /content/soccertrack.zip -d /content/soccertrack

  !rm /content/soccertrack.zip

In [ ]:
# Dataset processing
if rebuild_dataset:
  !pip install pandas pillow tqdm opencv-python -q

  frame_out_root = "/content/data_prep/soccertrack_frames"
  video_root = "/content/soccertrack/top_view/videos"
  annotation_root = "/content/soccertrack/top_view/annotations"
  out_root = "/content/data_prep/soccertrack_coco"
  train_split = 0.8
  frame_stride = 20

  if not os.path.isdir(frame_out_root):
    print(f'Extract the frames from mp4 files {video_root} to {frame_out_root}')
    extract_frames_from_videos(video_root=video_root, output_root=frame_out_root, frame_stride=frame_stride)

  if not os.path.isdir(out_root):
    print(f'Convert the SoccerTrack CSVs to COCO format')
    convert_soccertrack_csvs_to_coco(
        annotation_root=annotation_root, image_root=frame_out_root, output_root=out_root, train_split=train_split, frame_stride=frame_stride
    )


In [ ]:
if visualize_sample:
  !pip install matplotlib -q
  from visualization import visualize_coco_sample
  visualize_coco_sample(
    coco_json_path="/content/soccertrack_coco/annotations/train.json",
    image_root="/content/soccertrack_coco/images/train"
  )

In [ ]:
device = torch.cuda.is_available()
print(f"GPU device for training / inference: {device}")